In [1]:
from tkinter.messagebox import NO
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import time
import scipy.io as scio

seed = 1234
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class Net(nn.Module):
    # Initialize the class
    def __init__(self):
        super(Net, self).__init__()
        self.hidden_layer1 = nn.Linear(2,20)
        self.hidden_layer2 = nn.Linear(20,20)
        self.hidden_layer3 = nn.Linear(20,20)
        self.hidden_layer4 = nn.Linear(20,20)
        self.hidden_layer5 = nn.Linear(20,20)
        self.hidden_layer6 = nn.Linear(20,20)
        self.hidden_layer7 = nn.Linear(20,20)
        self.hidden_layer8 = nn.Linear(20,20)
        self.hidden_layer9 = nn.Linear(20,20)
        self.output_layer = nn.Linear(20,2)

        for layer in [self.hidden_layer1, self.hidden_layer2, self.hidden_layer3, 
                      self.hidden_layer4, self.hidden_layer5, self.hidden_layer6, 
                      self.hidden_layer7, self.hidden_layer8, self.hidden_layer9, 
                      self.output_layer]:
            init.xavier_uniform_(layer.weight)
            init.zeros_(layer.bias)

    def forward(self, x, y):
        inputs = torch.cat([x, y], axis = 1)
        layer1_out = torch.tanh(self.hidden_layer1(inputs))
        layer2_out = torch.tanh(self.hidden_layer2(layer1_out))
        layer3_out = torch.tanh(self.hidden_layer3(layer2_out))
        layer4_out = torch.tanh(self.hidden_layer4(layer3_out))
        layer5_out = torch.tanh(self.hidden_layer5(layer4_out))
        layer6_out = torch.tanh(self.hidden_layer6(layer5_out))
        layer7_out = torch.tanh(self.hidden_layer7(layer6_out))
        layer8_out = torch.tanh(self.hidden_layer8(layer7_out))
        layer9_out = torch.tanh(self.hidden_layer9(layer8_out))
        output = self.output_layer(layer9_out)
        return output
        
    def net_NS(self, x, y):
        w = self.forward(x, y)
        
        u = w[:,0:1]
        v = w[:,1:2]
        
        return [u, v]
    
    def net_diff(self, x, y):
        w = self.forward(x, y)
        
        u = w[:,0:1]
        v = w[:,1:2]
        
        u_x = torch.autograd.grad(u.sum(), x, create_graph = True)[0] 
        u_y = torch.autograd.grad(u.sum(), y, create_graph = True)[0]
        v_x = torch.autograd.grad(v.sum(), x, create_graph = True)[0]
        v_y = torch.autograd.grad(v.sum(), y, create_graph = True)[0]

        return [u_x, u_y, v_x, v_y] 

    def predict(self, x, y):
        return self.net_diff(x, y), self.net_NS(x,y)
        

def closure():
    if torch.is_grad_enabled():
        adam_optimizer.zero_grad()
    outputs = net(x_device, y_device)
    l = loss_fn(outputs, targets)
    if l.requires_grad:
        l.backward()
    return l

net = Net()
net = Net().to(device)
loss_fn = nn.MSELoss()
adam_optimizer = torch.optim.Adam(net.parameters())
lbfgs_optimizer = torch.optim.LBFGS(net.parameters(), lr=1, max_iter=50000, max_eval=50000, history_size=50, line_search_fn="strong_wolfe")
ITERATIONS = 20000

# Load Data
train_data = np.loadtxt(f'data/TecGrid-Cavity_1_0.01.dat', delimiter=' ', skiprows = 3)
x = train_data[:,0:1]
y = train_data[:,1:2]

u = train_data[:,3:4]
v = train_data[:,4:5]
u_device = torch.from_numpy(u).float().to(device)
v_device = torch.from_numpy(v).float().to(device)
targets = torch.cat([u_device, v_device], axis = 1)


for epoch in range(ITERATIONS):
    adam_optimizer.zero_grad()

    x_device = torch.from_numpy(x).float().to(device)
    x_device.requires_grad = True
    y_device = torch.from_numpy(y).float().to(device)
    y_device.requires_grad = True

    output = net(x_device, y_device)
    # output = torch.cat([u_out, v_out], axis = 1)
    loss = loss_fn(output, targets)
    loss.backward()
    adam_optimizer.step()

    with torch.autograd.no_grad():
    	if (epoch+1) % 100 == 0:
            print(epoch+1,"Adam Loss:",loss.data)

last_loss = 0
for i in range(50000):
    lbfgs_optimizer.step(closure)
    with torch.no_grad():
        current_loss = closure()
        print(i, "L-BFGS-B Loss:", current_loss.item())
        if current_loss <= 1 * np.finfo(float).eps:
            break
        if current_loss == last_loss:
            break
        last_loss = current_loss
print("Optimization finished.")


100 Adam Loss: tensor(4.8491, device='cuda:0')
200 Adam Loss: tensor(1.8741, device='cuda:0')
300 Adam Loss: tensor(1.0215, device='cuda:0')
400 Adam Loss: tensor(0.6425, device='cuda:0')
500 Adam Loss: tensor(0.3702, device='cuda:0')
600 Adam Loss: tensor(0.2117, device='cuda:0')
700 Adam Loss: tensor(0.1280, device='cuda:0')
800 Adam Loss: tensor(0.0879, device='cuda:0')
900 Adam Loss: tensor(0.0644, device='cuda:0')
1000 Adam Loss: tensor(0.0477, device='cuda:0')
1100 Adam Loss: tensor(0.0376, device='cuda:0')
1200 Adam Loss: tensor(0.0305, device='cuda:0')
1300 Adam Loss: tensor(0.0258, device='cuda:0')
1400 Adam Loss: tensor(0.0217, device='cuda:0')
1500 Adam Loss: tensor(0.0183, device='cuda:0')
1600 Adam Loss: tensor(0.0159, device='cuda:0')
1700 Adam Loss: tensor(0.0140, device='cuda:0')
1800 Adam Loss: tensor(0.0124, device='cuda:0')
1900 Adam Loss: tensor(0.0121, device='cuda:0')
2000 Adam Loss: tensor(0.0101, device='cuda:0')
2100 Adam Loss: tensor(0.0095, device='cuda:0')
2

In [4]:
result_dif, result = net.predict(x_device, y_device)

In [8]:
# 将Tensor转换为NumPy数组
u_numpy = result[0].cpu().detach().numpy()
v_numpy = result[1].cpu().detach().numpy()

u_re = abs((u_numpy - u)/u)
v_re = abs((v_numpy - v)/v)

write_data = np.column_stack((x, y, u_numpy, v_numpy, u, v, u_re, v_re))

np.savetxt('data/velocity_comparison.dat', write_data, delimiter=' ', 
           header='variables="x","y","u_predict","v_predict","u","v","u_re","v_re"', comments='', fmt='%f')

In [10]:
ux = result_dif[0].cpu().detach().numpy()
uy = result_dif[1].cpu().detach().numpy()
vx = result_dif[2].cpu().detach().numpy()
vy = result_dif[3].cpu().detach().numpy()

gradient = np.column_stack((ux, uy, vx, vy))
np.savetxt('data/square_cavity_flow.dat', write_data, delimiter=' ', comments='', fmt='%f')

In [ ]:
def plot_solution(X_star, u_star, index):
    
    lb = X_star.min(0)
    ub = X_star.max(0)
    nn = 200
    x = np.linspace(lb[0], ub[0], nn)
    y = np.linspace(lb[1], ub[1], nn)
    X, Y = np.meshgrid(x,y)
    
    U_star = griddata(X_star, u_star.flatten(), (X, Y), method='cubic')
    
    plt.figure(index)
    plt.pcolor(X,Y,U_star, cmap = 'jet')
    plt.colorbar()
    
    
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/4
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)

In [7]:
    N = y.shape[0]
    T = t.shape[0]
    
    YY = np.tile(y[:,None], (1,T)) # N x T
    TT = np.tile(t[:,None], (1,N)).T # N x T

    y = YY.flatten()[:,None] # NT x 1
    t = TT.flatten()[:,None] # NT x 1
    rho = rho.flatten()[:,None] # NT x 1
    
    model = PhysicsInformedNN(y, t, rho, layers)
    model.train(iter_times)

    # Prediction
    rho_pred, fdiff_pred = model.predict(y, t)

    print(np.array(fdiff_pred).shape)
    
    scio.savemat(f'data/Diffusion_flow_new.mat', {'rho':rho, 'rho_t':fdiff_pred[0], 'rho_y':fdiff_pred[1], 'rho_yy':fdiff_pred[2], 'rho_3y':fdiff_pred[3]})

    # Error
    error_rho = np.linalg.norm(rho-rho_pred,2)/np.linalg.norm(rho,2)
    a =  (rho-rho_pred)/rho
    abs_error_rho = np.mean(abs(a))
 
    print('Error rho: %e' % (error_rho)) 
    print('Abs error rho: %e' % (abs_error_rho))


AttributeError: module 'tensorflow' has no attribute 'Session'